In [52]:
import numpy as np
import importlib
import tweepy
import datetime
import pandas as pd
import re
import pickle
import matplotlib.pyplot as plt
from tweepy import OAuthHandler
import funcs as fs
import requests
# dane do autoryzacji
consumer_key, consumer_secret ,access_token, access_secret = list(pd.read_csv('tw_auth.csv', header=None)[0])

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit_notify = True, wait_on_rate_limit=True)

### Funkcje z pliku - trzeba reloadowac po zmianach

In [58]:
importlib.reload(fs)

<module 'funcs' from 'C:\\Users\\Mag\\Documents\\_projekty\\2018 11 Twitter\\funcs.py'>

In [4]:
fs.get_state('Lublin, POlska')

'lubelskie'

Wczytanie tweetów ściągniętych wcześniej.

In [5]:
with open('all_tweets.pickle','rb') as f:
    geo_df= pickle.load(f)

In [17]:
geo_df.head(10)

,tw_text,location,user_id,user_name,fav_cnt,friends_cnt,foll_cnt
1,. Chyba się upiłam :),Lublin,488608743,Krysia,26257,254,186
5,@bakistaki *podchodzą do stolika i Tony odsuwa...,"Stark Tower, NY",960582252,Tony Stark,24431,127,349
9,marnie,"chlo,jess,ana,jess,livvy",4439269169,nic appreciates marnie💛,120621,1332,2769
11,wy je ba ne,"Bydgoszcz, Poland",979486037956354048,𝐚𝐝𝐚 ♡,67318,165,392
12,dokończę odcinek i idę spać,"Oslo, Norge",3825344361,بشرة,3007,230,1058
16,To jest chyba najpiękniejsze koncertowe zdjęci...,"Czestochowa, Polonia",93217088,★ 𝑴𝒐𝒏𝒊𝒌𝒂 ★,9434,525,833
17,Jeśli kogoś nie ma na hali.\nLive + możliwość ...,Katowice,127498860,czaik 🇵🇱,2638,598,236
18,STRONE Z KROLIKAMI USUNELI BO BYL NIELEGALNY H...,My Own World,795642921437593601,Prevlingerka 😇🌙✈,8960,628,658
19,@SEIBA11337 Noo ale what XDDD czemu w mojej dz...,Driver Kim squad,952329448812109825,another story spoilers,11452,148,214
20,"""Marzenia są jak zapach mgły o poranku, po pew...","Dębica, Polska",980483662713106434,Piotrek,170,89,56


#### Usuwanie lokalizacji, dla których nie ma sensu szukać 
Dodane później do samej funkcji

In [18]:
bad_locs = ['', 'polska', 'poland','europe', 'europa']

In [130]:
geo_df['tw_location'][10].lower()

'eu🇪🇺, poland'

In [20]:
for i in geo_df.index:
    if geo_df['location'][i].lower() in bad_locs:
        geo_df=geo_df.drop([i])
#geo_df['tw_location'] not in bad_locs

In [143]:
len(geo_df)
##5956

5956

### Testowanie funkcji do ściągania lokalizacji
Funkcja w pliku funcs

In [99]:
def get_state(location):
    location = re.sub('\W', '',location)
    conf = {'format': 'json', 'addressdetails': 1, 'limit' : 1, 'q': location}
    try:
        loc_data = requests.get('http://nominatim.openstreetmap.org', params=conf).json()
    except Exception as e:
        print(e.args + ('Couldn\'t get location for {}'.format(location)))
  
    try:
        state = loc_data[0]['address']['state']
    except KeyError:
        print('Location "{}" doesn\'t have a state value'.format(location))
        return None
    except IndexError:
        print('Couldn\'t get location for {}'.format(location))
        return None
    
    return state

In [61]:
fs.get_state('Podkarpackie, Polska')

'podkarpackie'

Ręczne oglądanie wyników

In [68]:
payload = {'format': 'json',

           'addressdetails': 1,

           'limit' : 1,
          }

payload['q'] = location

 

r = requests.get('http://nominatim.openstreetmap.org', params=payload)



#r.json()
r.url

'https://nominatim.openstreetmap.org/?format=json&addressdetails=1&limit=1&q=W+twoim+sercu'

In [107]:
u = r.json()

In [93]:
print(u)

[{'place_id': '198010785', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': '49715', 'boundingbox': ['49.0020468', '55.0336963', '14.1229707', '24.145783'], 'lat': '52.0977181', 'lon': '19.0258159', 'display_name': 'RP', 'class': 'boundary', 'type': 'administrative', 'importance': 0.89229898543353, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png', 'address': {'country': 'RP', 'country_code': 'pl'}}]


In [94]:
u[0]['address']['state']

KeyError: 'state'

In [24]:
r.url

'https://nominatim.openstreetmap.org/?format=json&addressdetails=1&limit=1&q=Warszawa'

In [75]:
rr = r.json()
type(rr)

list

In [36]:
rr[0]['address']['state']

'mazowieckie'

In [74]:
type(rr[0]['address'])

dict

In [43]:
geo_df.head(100)

,tw_text,location,user_id,user_name,fav_cnt,friends_cnt,foll_cnt,state
1,. Chyba się upiłam :),Lublin,488608743,Krysia,26257,254,186,lubelskie
5,@bakistaki *podchodzą do stolika i Tony odsuwa...,"Stark Tower, NY",960582252,Tony Stark,24431,127,349,New York
9,marnie,"chlo,jess,ana,jess,livvy",4439269169,nic appreciates marnie💛,120621,1332,2769,Missouri
11,wy je ba ne,"Bydgoszcz, Poland",979486037956354048,𝐚𝐝𝐚 ♡,67318,165,392,kujawsko-pomorskie
12,dokończę odcinek i idę spać,"Oslo, Norge",3825344361,بشرة,3007,230,1058,None
16,To jest chyba najpiękniejsze koncertowe zdjęci...,"Czestochowa, Polonia",93217088,★ 𝑴𝒐𝒏𝒊𝒌𝒂 ★,9434,525,833,śląskie
17,Jeśli kogoś nie ma na hali.\nLive + możliwość ...,Katowice,127498860,czaik 🇵🇱,2638,598,236,śląskie
18,STRONE Z KROLIKAMI USUNELI BO BYL NIELEGALNY H...,My Own World,795642921437593601,Prevlingerka 😇🌙✈,8960,628,658,California
19,@SEIBA11337 Noo ale what XDDD czemu w mojej dz...,Driver Kim squad,952329448812109825,another story spoilers,11452,148,214,Maryland
20,"""Marzenia są jak zapach mgły o poranku, po pew...","Dębica, Polska",980483662713106434,Piotrek,170,89,56,podkarpackie


In [31]:
#del geo_df['state']
#geo_df['state']=''

### Przypisywanie lokalizacji 
Do zmiany - dla dużych zbiorów zupełnie bez sensu.

Zamiast apply jest pętla, bo błędy wywalały cały update.

In [159]:
for i in geo_df[geo_df.state==''].index:
    geo_df.state.loc[i] = fs.get_state(geo_df.location.loc[i])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Analiza błędów.

In [130]:
location=geo_df.location.loc[1771]
print(location)
location = re.sub('\W', ' ',location)
print("\"", location, "\"")
conf = {'format': 'json', 'addressdetails': 1, 'limit' : 1, 'q': location}
loc_data = requests.get('http://nominatim.openstreetmap.org', params=conf).json()
print(len(loc_data), len(location.split()))
if ((len(loc_data)==0) & (len(location.split())>0)) :
    conf = {'format': 'json', 'addressdetails': 1, 'limit' : 1, 'q': location.split()[0]}
    loc_data = requests.get('http://nominatim.openstreetmap.org', params=conf).json()

    if len(loc_data)==0:
        print('Couldn\'t get location for {}'.format(location))


W twoim sercu
" W twoim sercu "
0 3


In [134]:
geo_df[geo_df.state==''].index

Int64Index([7007, 7008, 7009, 7010, 7015, 7016, 7018, 7020, 7021, 7023,
            ...
            9969, 9975, 9976, 9982, 9984, 9985, 9990, 9992, 9997, 9998],
           dtype='int64', length=1155)

In [136]:
'Warszawa, Polska'.split()[0]

'Warszawa,'

In [68]:
re.sub('\W', ' ', 'Rzeczpospolita ∞')

'Rzeczpospolita  '

In [69]:
re.sub('\W', ' ', 'Zamość ∞')

'Zamość  '

In [80]:
type(geo_full)

pandas.core.frame.DataFrame

### Tworzenie słownika z odczytanych już wartości

In [149]:
t = geo_df[['location', 'state']][geo_df.state!='']
t = t.dropna()
t = t.drop_duplicates()
len(t)
#1322

1636

In [150]:
#t=t.drop_duplicates()
t=pd.Series(t.state.values, index = t.location).to_dict()


In [151]:
for i in geo_df[geo_df.state==''].index:
    if geo_df.location.loc[i] in t.keys():
        geo_df.state.loc[i] = t[geo_df.location.loc[i]]

### Ile jeszcze brakuje?

In [153]:
len(geo_df[geo_df.state==''])

225

In [152]:
len(geo_df[geo_df.state!=''])
#1875
#3320
#3827
#3972
#4908
#5731

5731

Gdzie są braki?

In [128]:
geo_df[geo_df.state=='']

,tw_text,location,user_id,user_name,fav_cnt,friends_cnt,foll_cnt,state
4529,bangtani mają jedne z najlepszych stages na mm...,btshinee | animanga • design student,598257378,olecka,1673,127,7428,
4533,@keram_iksmodar Nasz największy przyjaciel czy...,PL,1066836781851705344,Madzik,817,533,91,
4534,"@cahadaee rozumiem i mam nadzieję, ze ci się p...",my bed,830093369820995585,julss,4314,562,753,
4536,@PrettyRoosie @vitae_dolorum Tymczasem siostry...,"Oslo, Norway",993581193621843968,𝕨𝕖𝕣𝕠𝕟𝕚𝕜𝕒❄24𝒹𝒶𝓎𝓈,177,19,3,
4543,@xukunpl Dam mu szansę na rehabilitację.,Band-Maid,4691831532,Saiki,229,297,161,
4546,Bam bang - Jessie J,"DKI Jakarta, Indonesia",82817687,Yusuf Bachtiar,702,849,2577,
4548,DROGA #KRZYŻOWA w #Koloseum 2005 – tekst kardy...,"Polska, Dobiegniew",221685539,ksPiotr 🇵🇱💯,5439,1837,1472,
4551,Nie mają żadnego senswowrgo argumentu żeby że ...,treat people with kindness,1901251202,ήλιος 🕊,598,566,467,
4554,"zrobilam sobie maseczke na wagry, juz mam ich ...",serdusio jina,941677941557252096,an. stream epiphany&awake📌,12239,342,285,
4557,"@Saiki_pl Jestem spokojny, ale no gdzie??",𝐚𝐮!//nsfw//29,1068213732919164928,蔡徐坤,72,129,72,


### Zrzucam to co jest do pickla i pzetwarzam dalej

In [155]:
geo_full = geo_df[(geo_df.state!='') & (geo_df.state!='None')]
geo_full=geo_full.dropna()

In [156]:
with open('geo.pickle','wb') as f:
    pickle.dump(geo_full,f)